This tutorial shows the multi-sectoral capabilities of Macro with synthetic natural gas (SNG) resource. Instead of using a natural gas source node to purchase natural gas for power and hydrogen generators, here a new natural gas (NG) node with demand input from csv file is created. The resources utilized used are PV power, electrolyzer H2, direct air capture (DAC) of CO2, and SNG plant. The system will uses electricity, hydrogen, and atmospheric CO2 to produce SNG to fulfill the NG demand. 

In [37]:
using Pkg
Pkg.activate(".")

  Activating project at `c:\Users\pecci\Code\MACRO-dev\tutorials`


In [38]:
using Macro,CSV,DataFrames

Start MACRO model generation (units: MWh for energy, $ for costs)

In [39]:
T = 10*24;# 8760;
macro_settings = (Commodities = Dict(Electricity=>Dict(:HoursPerTimeStep=>1,:HoursPerSubperiod=>T),
                                    Hydrogen=>Dict(:HoursPerTimeStep=>1,:HoursPerSubperiod=>T),
                                    NaturalGas=>Dict(:HoursPerTimeStep=>1,:HoursPerSubperiod=>T),
                                    CO2=>Dict(:HoursPerTimeStep=>1,:HoursPerSubperiod=>T)),
                PeriodLength = T);

In [40]:
macro_settings

(Commodities = Dict{DataType, Dict{Symbol, Int64}}(Electricity => Dict(:HoursPerSubperiod => 240, :HoursPerTimeStep => 1), NaturalGas => Dict(:HoursPerSubperiod => 240, :HoursPerTimeStep => 1), CO2 => Dict(:HoursPerSubperiod => 240, :HoursPerTimeStep => 1), Hydrogen => Dict(:HoursPerSubperiod => 240, :HoursPerTimeStep => 1)), PeriodLength = 240)

In [41]:

H2_MWh = 33.33 # MWh per tonne of H2
NG_MWh = 0.29307107 # MWh per MMBTU of NG

df = CSV.read("time_series_data_for_synthetic_natural_gas_tutorial.csv",DataFrame)

electricity_demand = df[1:T,:Electricity_Demand_MW]; # MWh
solar_capacity_factor = df[1:T,:Solar_Capacity_Factor]; # factor between 0 and 1
h2_demand = H2_MWh*df[1:T,:H2_Demand_tonne]; # MWh of hydrogen
ng_demand = NG_MWh*df[1:T,:NG_Demand_MMBtu]; # MWh of natural gas

solar_inv_cost = 85300; # $/MW
solar_fom_cost = 18760.0; # $/MW

battery_inv_cost = 19584.0; #$/MW
battery_fom_cost = 4895; # $/MW
battery_vom_cost = 0.15; #$/MW
battery_inv_cost_storage = 22494.0; #$/MWh
battery_fom_cost_storage = 5622; #$/MWh
battery_vom_cost_storage = 0.15; #$/MWh
battery_eff_up = 0.92;
battery_eff_down = 0.92;

electric_dac_inv_cost = 9390;# $/Tonnes of CO2 captured/h
electric_dac_fom_cost = 7470;# $/Tonnes of CO2 captured/h
electric_dac_vom_cost = 22; #$/Tonnes of CO2 captured
electric_dac_power_input = 4.38; #MWh/Tonnes of CO2 captured
electric_dac_capsize = 20;# Tonnes of CO2 captured/h

sng_inv_cost = 390;# $/MW of NG
sng_fom_cost = 470;# $/MW of NG
sng_vom_cost = 2; #$/MWh of NG
sng_power_input = 2; #MWh of power/MWh of NG
sng_h2_input = 2; #MWh of H2/MWh of NG
sng_co2_input = 2; #Tonnes of CO2 captured/MWh of NG
sng_capsize = 20;# MW of NG

electrolyzer_capsize = 2*H2_MWh; # MWh of H2
electrolyzer_efficiency = 1/(45/H2_MWh); # MWh of H2 / MWh of electricity
electrolyzer_inv_cost = 2033333/H2_MWh; # $/MW of H2
electrolyzer_fom_cost = 30500/H2_MWh; # $/MW of H2
electrolyzer_vom_cost = 0.0;


In [42]:
### For now, we use the same temporal resolution and subperiods for every commodity, but MACRO will allow to use different temporal resolution for different commodities

hours_per_timestep(c) = macro_settings.Commodities[c][:HoursPerTimeStep];
hours_per_subperiod(c) = macro_settings.Commodities[c][:HoursPerSubperiod]
time_interval(c)= 1:hours_per_timestep(c):macro_settings.PeriodLength

subperiods(c) = collect(Iterators.partition(time_interval(c), Int(hours_per_subperiod(c) / hours_per_timestep(c))),)

subperiods (generic function with 1 method)

Let's start from creating an electricity node:

In [43]:
e_node = Node{Electricity}(;
    id = Symbol("E_node"),
    demand =  electricity_demand,
    time_interval = time_interval(Electricity),
    subperiods = subperiods(Electricity),
    max_nsd = [0.0],
    price_nsd = [50000],
    constraints = [Macro.DemandBalanceConstraint(),Macro.MaxNonServedDemandConstraint()]
)

Node{Electricity}(:E_node, [7850.0, 7424.0, 7107.0, 6947.0, 6922.0, 7045.0, 7307.0, 7544.0, 7946.0, 8340.0  …  10262.0, 10332.0, 11008.0, 11778.0, 11691.0, 11401.0, 10973.0, 10250.0, 9297.0, 8468.0], 1:1:240, StepRange{Int64, Int64}[1:1:240], [0.0], [50000.0], Dict{Any, Any}(), Dict{Any, Any}(), Dict{DataType, Float64}(), Dict{DataType, Float64}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)])

We assign a solar PV generator to this electricity node:

In [44]:
solar_pv = Transformation{Electricity}(;
    id = :SolarPV,
    time_interval = time_interval(Electricity),
    subperiods = subperiods(Electricity),
    #### Note that this transformation does not have a stoichiometry balance because the sunshine is exogenous :-)
)
solar_pv.TEdges[:E] = TEdge{Electricity}(;
id = :E,
node = e_node,
transformation = solar_pv,
time_interval = time_interval(Electricity),
subperiods = subperiods(Electricity),
direction = :output,
has_planning_variables = true,
can_expand = true,
can_retire = false,
existing_capacity = 0.0,
capacity_factor = solar_capacity_factor,
investment_cost = solar_inv_cost,
fixed_om_cost = solar_fom_cost,
constraints = [Macro.CapacityConstraint()]
)

TEdge{Electricity}(:E, Node{Electricity}(:E_node, [7850.0, 7424.0, 7107.0, 6947.0, 6922.0, 7045.0, 7307.0, 7544.0, 7946.0, 8340.0  …  10262.0, 10332.0, 11008.0, 11778.0, 11691.0, 11401.0, 10973.0, 10250.0, 9297.0, 8468.0], 1:1:240, StepRange{Int64, Int64}[1:1:240], [0.0], [50000.0], Dict{Any, Any}(), Dict{Any, Any}(), Dict{DataType, Float64}(), Dict{DataType, Float64}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)]), Transformation{Electricity}(:SolarPV, 1:1:240, StepRange{Int64, Int64}[1:1:240], Symbol[], Dict{Symbol, TEdge}(:E => TEdge{Electricity}(#= circular reference @-3 =#)), Dict{Any, Any}(), Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[], 0.0, Inf, 0.0, false, false, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, :.), :output, true, false, true, 1.0, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1779, 0.429  …  0.5176, 0.3503, 0.1105, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 1:1:

We can also assigne a battery storage to the node:

In [45]:
battery = Transformation{Storage}(;
id = :battery,
stoichiometry_balance_names = [:storage],
time_interval = time_interval(Electricity),
subperiods = subperiods(Electricity),
can_expand = true,
can_retire = false,
existing_capacity_storage = 0.0,
investment_cost_storage = battery_inv_cost_storage,
fixed_om_cost_storage = battery_fom_cost_storage,
min_duration = 1,
max_duration = 10,
constraints = [Macro.StorageCapacityConstraint(),Macro.StoichiometryBalanceConstraint()],
discharge_capacity_edge = :discharge
)

battery.TEdges[:discharge] = TEdge{Electricity}(;
id = :discharge,
time_interval = time_interval(Electricity),
subperiods = subperiods(Electricity),
node = e_node,
transformation = battery,
direction = :output,
has_planning_variables = true,
can_expand = true,
can_retire = false,
existing_capacity = 0.0,
investment_cost = battery_inv_cost,
fixed_om_cost = battery_fom_cost,
variable_om_cost = battery_vom_cost,
st_coeff = Dict(:storage=>1/battery_eff_down),
constraints = [Macro.CapacityConstraint()]
)

battery.TEdges[:charge] = TEdge{Electricity}(;
id = :charge,
time_interval = time_interval(Electricity),
subperiods = subperiods(Electricity),
node = e_node,
transformation = battery,
direction = :input,
has_planning_variables = false,
st_coeff = Dict(:storage=>battery_eff_up)
)


TEdge{Electricity}(:charge, Node{Electricity}(:E_node, [7850.0, 7424.0, 7107.0, 6947.0, 6922.0, 7045.0, 7307.0, 7544.0, 7946.0, 8340.0  …  10262.0, 10332.0, 11008.0, 11778.0, 11691.0, 11401.0, 10973.0, 10250.0, 9297.0, 8468.0], 1:1:240, StepRange{Int64, Int64}[1:1:240], [0.0], [50000.0], Dict{Any, Any}(), Dict{Any, Any}(), Dict{DataType, Float64}(), Dict{DataType, Float64}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)]), Transformation{Storage}(:battery, 1:1:240, StepRange{Int64, Int64}[1:1:240], [:storage], Dict{Symbol, TEdge}(:charge => TEdge{Electricity}(#= circular reference @-3 =#), :discharge => TEdge{Electricity}(:discharge, Node{Electricity}(:E_node, [7850.0, 7424.0, 7107.0, 6947.0, 6922.0, 7045.0, 7307.0, 7544.0, 7946.0, 8340.0  …  10262.0, 10332.0, 11008.0, 11778.0, 11691.0, 11401.0, 10973.0, 10250.0, 9297.0, 8468.0], 1:1:240, StepRange{Int64, Int64}[1:1:240], [0.0], [500

We also consider a hydrogen demand node:

In [46]:
h2_node = Node{Hydrogen}(;
    id = Symbol("H2_node"),
    demand =  h2_demand,
    time_interval = time_interval(Hydrogen),
    subperiods = subperiods(Hydrogen),
    max_nsd = [0.0],
    price_nsd = [0.0],
    constraints = [Macro.DemandBalanceConstraint()]
)

Node{Hydrogen}(:H2_node, [53.9946, 40.6626, 58.3275, 133.32, 274.6392, 446.95529999999997, 559.6107, 641.2692, 674.5991999999999, 714.2619  …  820.5846, 871.2461999999999, 846.9153, 796.2537, 655.9344, 481.6184999999999, 348.6318, 262.3071, 155.65109999999999, 87.9912], 1:1:240, StepRange{Int64, Int64}[1:1:240], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Dict{DataType, Float64}(), Dict{DataType, Float64}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing)])

We also consider a natural gas demand node:

In [47]:
ng_node = Node{NaturalGas}(;
    id = Symbol("NG_node"),
    demand =  ng_demand,
    time_interval = time_interval(NaturalGas),
    subperiods = subperiods(NaturalGas),
    max_nsd = [0.0],
    price_nsd = [0.0],
    constraints = [Macro.DemandBalanceConstraint()]
)

Node{NaturalGas}(:NG_node, [0.4747751334, 0.3575467054, 0.5128743725, 1.17228428, 2.4149056168, 3.9300830487, 4.9206632653, 5.638687386799999, 5.931758456799999, 6.2805130301  …  7.2154097434, 7.6608777698, 7.4469358887, 7.0014678623, 5.7676386576, 4.2348769614999995, 3.0655233922000003, 2.3064693209, 1.3686418968999998, 0.7737076248], 1:1:240, StepRange{Int64, Int64}[1:1:240], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Dict{DataType, Float64}(), Dict{DataType, Float64}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing)])

We also define a CO2 node:

In [48]:
co2_node = Node{CO2}(;
id = Symbol("CO2_node"),
time_interval = time_interval(CO2),
subperiods = subperiods(CO2),
demand = zeros(length(time_interval(CO2))),
max_nsd = [0.0],
price_nsd = [0.0],
)

Node{CO2}(:CO2_node, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 1:1:240, StepRange{Int64, Int64}[1:1:240], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Dict{DataType, Float64}(), Dict{DataType, Float64}(), Macro.AbstractTypeConstraint[])

We define captured CO2 node and sink to account for the inventory of captured CO2 (input from CO2 capture, output to SNG or potential storage):

In [49]:
co2_captured_node = Node{CO2Captured}(;
id = Symbol("CO2_captured_node"),
time_interval = time_interval(CO2),
subperiods = subperiods(CO2),
demand = zeros(length(time_interval(CO2))),
max_nsd = [0.0],
price_nsd = [0.0],
constraints = [Macro.DemandBalanceConstraint()]
)

co2_captured_sink = Transformation{CO2Captured}(;
id = :co2_captured_sink,
time_interval = time_interval(CO2),
subperiods = subperiods(CO2),
#### Note that this transformation does not have a stoichiometry balance because we are modeling just a sink
)

co2_captured_sink.TEdges[:co2_captured_flow] = TEdge{CO2Captured}(;
id = :co2_sink_flow,
node = co2_captured_node,
transformation = co2_captured_sink,
direction = :input,
time_interval = time_interval(CO2),
subperiods = subperiods(CO2),
has_planning_variables = false,
)


TEdge{CO2Captured}(:co2_sink_flow, Node{CO2Captured}(:CO2_captured_node, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 1:1:240, StepRange{Int64, Int64}[1:1:240], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Dict{DataType, Float64}(), Dict{DataType, Float64}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing)]), Transformation{CO2Captured}(:co2_captured_sink, 1:1:240, StepRange{Int64, Int64}[1:1:240], Symbol[], Dict{Symbol, TEdge}(:co2_captured_flow => TEdge{CO2Captured}(#= circular reference @-3 =#)), Dict{Any, Any}(), Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[], 0.0, Inf, 0.0, false, false, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, :.), :input, false, false, false, 1.0, Float64[], 1:1:240, StepRange{Int64, Int64}[1:1:240], Dict{Symbol, Float64}(), 0.0, Inf, 0.0, 0.0, 0.0, 0.0, Float64[], 0.0, false, 1.0, 1.0, 0, 0, 0.0, Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeC

We can also create an electrolyzer, which is defined by a single stoichiometry balance: electrolyzer_efficiency $\times$ Electricity = H2

In [50]:
electrolyzer = Transformation{Electrolyzer}(;
                id = :Electrolyzer,
                time_interval = time_interval(Electricity),
                stoichiometry_balance_names = [:energy],
                constraints = [Macro.StoichiometryBalanceConstraint()]
                )

electrolyzer.TEdges[:H2] = TEdge{Hydrogen}(;
    id = :H2,
    node = h2_node,
    transformation = electrolyzer,
    direction = :output,
    has_planning_variables = true,
    can_expand = true,
    can_retire = false,
    capacity_size = electrolyzer_capsize,
    time_interval = time_interval(Hydrogen),
    subperiods = subperiods(Hydrogen),
    st_coeff = Dict(:energy=>1.0),
    existing_capacity = 0.0,
    investment_cost = electrolyzer_inv_cost,
    fixed_om_cost = electrolyzer_fom_cost,
    variable_om_cost = electrolyzer_vom_cost,
    constraints = [Macro.CapacityConstraint()]
)

electrolyzer.TEdges[:E] = TEdge{Electricity}(;
    id = :E,
    node = e_node,
    transformation = electrolyzer,
    direction = :input,
    has_planning_variables = false,
    time_interval = time_interval(Electricity),
    subperiods = subperiods(Electricity),
    st_coeff = Dict(:energy=>electrolyzer_efficiency)
)

TEdge{Electricity}(:E, Node{Electricity}(:E_node, [7850.0, 7424.0, 7107.0, 6947.0, 6922.0, 7045.0, 7307.0, 7544.0, 7946.0, 8340.0  …  10262.0, 10332.0, 11008.0, 11778.0, 11691.0, 11401.0, 10973.0, 10250.0, 9297.0, 8468.0], 1:1:240, StepRange{Int64, Int64}[1:1:240], [0.0], [50000.0], Dict{Any, Any}(), Dict{Any, Any}(), Dict{DataType, Float64}(), Dict{DataType, Float64}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)]), Transformation{Electrolyzer}(:Electrolyzer, 1:1:240, StepRange{Int64, Int64}[], [:energy], Dict{Symbol, TEdge}(:H2 => TEdge{Hydrogen}(:H2, Node{Hydrogen}(:H2_node, [53.9946, 40.6626, 58.3275, 133.32, 274.6392, 446.95529999999997, 559.6107, 641.2692, 674.5991999999999, 714.2619  …  820.5846, 871.2461999999999, 846.9153, 796.2537, 655.9344, 481.6184999999999, 348.6318, 262.3071, 155.65109999999999, 87.9912], 1:1:240, StepRange{Int64, Int64}[1:1:240], [0.0], [0.0], Dict{An

Let's add a direct air capture unit consuming electricity and capturing CO2 from the atmosphere into the captured CO2 inventory

In [51]:
electric_dac = Transformation{DACElectric}(;
                id = :DAC,
                time_interval = time_interval(CO2),
                stoichiometry_balance_names  = [:energy,:captured_co2],
                constraints = [Macro.StoichiometryBalanceConstraint()]
                )

electric_dac.TEdges[:CO2_Captured] = TEdge{CO2Captured}(;
    id = :CO2_Captured,
    node = co2_captured_node,
    transformation = electric_dac,
    direction = :output,
    has_planning_variables = true,
    can_expand = true,
    can_retire = false,
    capacity_size = electric_dac_capsize,
    time_interval = time_interval(CO2),
    subperiods = subperiods(CO2),
    st_coeff = Dict(:energy=>electric_dac_power_input,:captured_co2=>1.0),
    existing_capacity = 0.0,
    investment_cost = electric_dac_inv_cost,
    fixed_om_cost = electric_dac_fom_cost,
    variable_om_cost = electric_dac_vom_cost,
    constraints = [Macro.CapacityConstraint()]
)

electric_dac.TEdges[:CO2_Input] = TEdge{CO2}(;
    id = :CO2_Input,
    node = co2_node,
    transformation = electric_dac,
    direction = :input,
    has_planning_variables = false,
    time_interval = time_interval(CO2),
    subperiods = subperiods(CO2),
    st_coeff = Dict(:energy=>0.0,:captured_co2=>1.0)
)

electric_dac.TEdges[:E] = TEdge{Electricity}(;
    id = :E,
    node = e_node,
    transformation = electric_dac,
    direction = :input,
    has_planning_variables = false,
    time_interval = time_interval(Electricity),
    subperiods = subperiods(Electricity),
    st_coeff = Dict(:energy=>1.0,:captured_co2=>0.0)
)

TEdge{Electricity}(:E, Node{Electricity}(:E_node, [7850.0, 7424.0, 7107.0, 6947.0, 6922.0, 7045.0, 7307.0, 7544.0, 7946.0, 8340.0  …  10262.0, 10332.0, 11008.0, 11778.0, 11691.0, 11401.0, 10973.0, 10250.0, 9297.0, 8468.0], 1:1:240, StepRange{Int64, Int64}[1:1:240], [0.0], [50000.0], Dict{Any, Any}(), Dict{Any, Any}(), Dict{DataType, Float64}(), Dict{DataType, Float64}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)]), Transformation{DACElectric}(:DAC, 1:1:240, StepRange{Int64, Int64}[], [:energy, :captured_co2], Dict{Symbol, TEdge}(:CO2_Captured => TEdge{CO2Captured}(:CO2_Captured, Node{CO2Captured}(:CO2_captured_node, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 1:1:240, StepRange{Int64, Int64}[1:1:240], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Dict{DataType, Float64}(), Dict{DataType, Float64}(), Macro.AbstractTyp

Let's add a synthetic natural gas unit consuming electricity, hydrogen, and captured CO2 to produce natural gas output

In [52]:
sng = Transformation{SyntheticNG}(;
                id = :SNG,
                time_interval = time_interval(NaturalGas),
                stoichiometry_balance_names  = [:electricity,:hydrogen,:co2],
                constraints = [Macro.StoichiometryBalanceConstraint()]
                )

sng.TEdges[:NG] = TEdge{NaturalGas}(;
    id = :NG,
    node = ng_node,
    transformation = sng,
    direction = :output,
    has_planning_variables = true,
    can_expand = true,
    can_retire = false,
    capacity_size = sng_capsize,
    time_interval = time_interval(NaturalGas),
    subperiods = subperiods(NaturalGas),
    st_coeff = Dict(:electricity=>sng_power_input,:hydrogen=>sng_h2_input,:co2=>sng_co2_input),
    existing_capacity = 0.0,
    investment_cost = sng_inv_cost,
    fixed_om_cost = sng_fom_cost,
    variable_om_cost = sng_vom_cost,
    constraints = [Macro.CapacityConstraint()]
)

sng.TEdges[:E] = TEdge{Electricity}(;
    id = :E,
    node = e_node,
    transformation = sng,
    direction = :input,
    has_planning_variables = false,
    time_interval = time_interval(Electricity),
    subperiods = subperiods(Electricity),
    st_coeff = Dict(:electricity=>1.0,:hydrogen=>0.0,:co2=>0.0)
)

        
sng.TEdges[:H2] = TEdge{Hydrogen}(;
    id = :H2,
    node = h2_node,
    transformation = sng,
    direction = :input,
    has_planning_variables = false,
    time_interval = time_interval(Hydrogen),
    subperiods = subperiods(Hydrogen),
    st_coeff = Dict(:electricity=>0.0,:hydrogen=>1.0,:co2=>0.0)
)

sng.TEdges[:CO2] = TEdge{CO2Captured}(;
    id = :CO2,
    node = co2_captured_node,
    transformation = sng,
    direction = :input,
    has_planning_variables = false,
    time_interval = time_interval(CO2),
    subperiods = subperiods(CO2),
    st_coeff = Dict(:electricity=>0.0,:hydrogen=>0.0,:co2=>1.0)
)



TEdge{CO2Captured}(:CO2, Node{CO2Captured}(:CO2_captured_node, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 1:1:240, StepRange{Int64, Int64}[1:1:240], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Dict{DataType, Float64}(), Dict{DataType, Float64}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing)]), Transformation{SyntheticNG}(:SNG, 1:1:240, StepRange{Int64, Int64}[], [:electricity, :hydrogen, :co2], Dict{Symbol, TEdge}(:NG => TEdge{NaturalGas}(:NG, Node{NaturalGas}(:NG_node, [0.4747751334, 0.3575467054, 0.5128743725, 1.17228428, 2.4149056168, 3.9300830487, 4.9206632653, 5.638687386799999, 5.931758456799999, 6.2805130301  …  7.2154097434, 7.6608777698, 7.4469358887, 7.0014678623, 5.7676386576, 4.2348769614999995, 3.0655233922000003, 2.3064693209, 1.3686418968999998, 0.7737076248], 1:1:240, StepRange{Int64, Int64}[1:1:240], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Dict{DataTyp

Now that we have implemented different technologies, we can create the JuMP model.

In [53]:
nodes = [e_node; h2_node; ng_node; co2_node; co2_captured_node];
system = [nodes;solar_pv; battery; electrolyzer; electric_dac; sng;co2_captured_sink];

In [54]:
model = Macro.JuMP.Model();
Macro.@variable(model,vREF==1) ## Variable used to initialize empty expressions
Macro.@expression(model, eFixedCost, 0 * model[:vREF]);
Macro.@expression(model, eVariableCost, 0 * model[:vREF]);

In [55]:
Macro.add_planning_variables!.(system,model);

In [56]:
Macro.add_operation_variables!.(system, model);

In [57]:
Macro.add_all_model_constraints!.(system, model);

┌ Warning: MaxNonServedDemandConstraint required for a node that does not have a non-served demand variable so MACRO will not create this constraint
└ @ Macro c:\Users\pecci\Code\MACRO-dev\src\node.jl:92


In [58]:
Macro.@objective(model, Min, model[:eFixedCost] + model[:eVariableCost]);


In [59]:
using Gurobi

In [60]:
Macro.set_optimizer(model,Gurobi.Optimizer)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-18


In [61]:
Macro.optimize!(model)

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i9-9980HK CPU @ 2.40GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 4089 rows, 3379 columns and 9473 nonzeros
Model fingerprint: 0x54806583
Coefficient statistics:
  Matrix range     [2e-02, 7e+01]
  Objective range  [1e-01, 4e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [4e-01, 1e+04]
Presolve removed 3087 rows and 2615 columns
Presolve time: 0.02s
Presolved: 1002 rows, 764 columns, 3694 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.1416290e+09   2.758429e+06   0.000000e+00      0s
     144    1.5371952e+10   0.000000e+00   0.000000e+00      0s

Solved in 144 iterations and 0.02 seconds (0.01 work units)
Optimal objective  1.537195208e+10

User-callback calls 208, time in user-callback 0.00 sec


The installed electrolyzer capacity is:

In [62]:
Macro.value(Macro.capacity(electrolyzer.TEdges[:H2]))

1057.8444733466

The installed solar capacity in MW is:

In [63]:
Macro.value(Macro.capacity(solar_pv.TEdges[:E]))

87461.27974606889

The installed battery capacity in MW is:

In [64]:
Macro.value(Macro.capacity(battery.TEdges[:discharge]))

20300.53671320273

The installed Electric DAC capacity in Tons CO2/h is:

In [65]:
Macro.value(Macro.capacity(electric_dac.TEdges[:CO2_Captured]))

18.2817733466

The installed SNG capacity in MW NG is:

In [66]:
Macro.value(Macro.capacity(sng.TEdges[:NG]))

9.1408866733

The resulting CO2 removed from the atmosphere by DAC in tonnes are:

In [67]:
atm_CO2_captured  = Macro.sum(Macro.value.(Macro.flow(electric_dac.TEdges[:CO2_Captured])))

2147.5486024817997

Which should be equal to the CO2 input to the electric DAC unit:

In [68]:
Macro.sum(Macro.value.(Macro.flow(electric_dac.TEdges[:CO2_Input])))

2147.5486024817997

And to the CO2 going into the SNG unit:

In [69]:
Macro.sum(Macro.value.(Macro.flow(sng.TEdges[:CO2])))

2147.5486024817997

The correct operation of DAC and SNG unit is enforced by the balance constraint at the captured CO2 node:

In [70]:
co2_captured_node.constraints[1].constraint_ref[5]

vFLOW_DAC_CO2_Captured[5] - vFLOW_SNG_CO2[5] - vFLOW_co2_captured_sink_co2_sink_flow[5] == 0

In [71]:
co2_node.constraints

Macro.AbstractTypeConstraint[]

In [72]:
Macro.net_balance(co2_node)[5]

-vFLOW_DAC_CO2_Input[5]